In [6]:
# Retrieve the current active GCP project from gcloud config
ACTIVE_PROJECT = !gcloud config get-value core/project
ACTIVE_PROJECT = ACTIVE_PROJECT[0]

# Specify region and GCS bucket path for pipeline artifacts
REGION_NAME = "us-central1"
PIPELINE_BUCKET = f"gs://{ACTIVE_PROJECT}-mlops-pipelines"



In [7]:
ACTIVE_PROJECT

'magnetic-guild-473821-t7'

Step 1 — Create the Dockerfile

In [8]:
%%writefile mlops-cli/Dockerfile

# Use Google's base image for CPU ML workloads
FROM gcr.io/deeplearning-platform-release/base-cpu

# Define working directory
WORKDIR /mlops-cli

# Copy your pipeline script and dependency list
COPY pipeline.py ./pipeline.py
COPY requirements.txt ./requirements.txt

# Install dependencies
RUN pip install --no-cache-dir -r requirements.txt

# Default command: execute the pipeline script
ENTRYPOINT ["python", "pipeline.py"]


Overwriting mlops-cli/Dockerfile


Enable Docker authentication with Artifact Registry

In [9]:
! gsutil mb -l {REGION_NAME} -p {ACTIVE_PROJECT} {PIPELINE_BUCKET}


Creating gs://magnetic-guild-473821-t7-mlops-pipelines/...
ServiceException: 409 A Cloud Storage bucket named 'magnetic-guild-473821-t7-mlops-pipelines' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


Enable Docker authentication with Artifact Registry

In [11]:
!gcloud auth configure-docker us-central1-docker.pkg.dev



{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
gcloud credential helpers already registered correctly.


Create a new Artifact Registry repository

In [12]:
DOCKER_REPO_NAME = "mlops-artifacts"


In [13]:
ACTIVE_PROJECT = ACTIVE_PROJECT.strip().replace("\n", "")


In [14]:
!gcloud artifacts repositories create $DOCKER_REPO_NAME \
    --repository-format=docker \
    --location=$REGION_NAME \
    --description="Repository for storing custom MLOps Docker images" \
    --project=$ACTIVE_PROJECT --quiet


ERROR: (gcloud.artifacts.repositories.create) ALREADY_EXISTS: the repository already exists


Define and verify image paths

In [15]:
# Define registry base and full image path
REGISTRY_HOST = f"{REGION_NAME}-docker.pkg.dev"
IMAGE_LABEL = "mlops-pipeline-image"
VERSION_TAG = "v1"
IMAGE_PATH = f"{REGISTRY_HOST}/{ACTIVE_PROJECT}/{DOCKER_REPO_NAME}/{IMAGE_LABEL}:{VERSION_TAG}"

# Confirm the final image URI
IMAGE_PATH


'us-central1-docker.pkg.dev/magnetic-guild-473821-t7/mlops-artifacts/mlops-pipeline-image:v1'

Build Docker image locally

In [16]:
! docker build -t mlops-pipeline:latest ./mlops-cli --quiet


sha256:f303592cc02c05b61bc9926ad9cb538c6ce4d6fe843e4252dfcb0aeb2ad32ed0


Tag the image with the Artifact Registry path

In [53]:
! docker tag mlops-pipeline:latest {IMAGE_PATH}

In [65]:
!pip install -q --upgrade pip

Alternative: Use Cloud Build for building and pushing

In [67]:
! gcloud builds submit --tag {IMAGE_PATH} --timeout=15m ./mlops-cli --quiet


Creating temporary archive of 6 file(s) totalling 22.3 KiB before compression.
Uploading tarball of [./mlops-cli] to [gs://magnetic-guild-473821-t7_cloudbuild/source/1759593571.374581-cc41a3cddce1446ca4ac3311413124b7.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/magnetic-guild-473821-t7/locations/global/builds/ceae479e-b640-48e1-b619-09d5fad90174].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/ceae479e-b640-48e1-b619-09d5fad90174?project=870469359193 ].
Waiting for build to complete. Polling interval: 1 second(s).
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "ceae479e-b640-48e1-b619-09d5fad90174"

FETCHSOURCE
Fetching storage object: gs://magnetic-guild-473821-t7_cloudbuild/source/1759593571.374581-cc41a3cddce1446ca4ac3311413124b7.tgz#1759593571584049
Copying gs://magnetic-guild-473821-t7_cloudbuild/source/1759593571.374581-cc41a3cddce1446ca4ac3311413124b7.tgz#1759593571584049...
/ [1 files][

Push image manually (if built locally)

In [63]:
! docker push {IMAGE_PATH}


The push refers to repository [us-central1-docker.pkg.dev/magnetic-guild-473821-t7/mlops-artifacts/mlops-pipeline-image]

99001ccd: Preparing 
bdbed5b1: Preparing 
972a6fdd: Preparing 
984a22b7: Preparing 
591c8231: Preparing 
a27eabb6: Preparing 
0ec5d133: Preparing 
bf18a086: Preparing 
fd623d1c: Preparing 
bf18a086: Preparing 
368dac1e: Preparing 
86ee7852: Preparing 
b840ebbc: Preparing 
3a398424: Preparing 
c8de4027: Preparing 
0471ea10: Preparing 
472bb674: Preparing 
6db02163: Preparing 
fae98bdb: Preparing 
4efe1a9f: Preparing 
b797ecd6: Preparing 
8bc36be9: Preparing 
7d7b828a: Preparing 
332fd020: Preparing 
721dfff5: Preparing 
e5180c54: Preparing 
7c40e807: Preparing 
f18a086: Preparing 
c68893aa: Preparing 
f18a086: Preparing 
0fadd3bc: Preparing 
cc8cc005: Preparing 
0fadd3bc: Layer already exists v1: digest: sha256:177d4bc3e8a041adbde58d392830b666e91ed862e1db5f190d89d25c4f223e91 size: 7835


(Alternative) Use a Cloud Build configuration file

In [69]:
! gcloud builds submit --config cloudbuild.yaml


Creating temporary archive of 278 file(s) totalling 249.4 MiB before compression.
Uploading tarball of [.] to [gs://magnetic-guild-473821-t7_cloudbuild/source/1759595753.624644-2f4f5eee86de4ee5ac0ba6b10c132d12.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/magnetic-guild-473821-t7/locations/global/builds/c2d73b3d-6771-449f-a8e9-3c458cb9c074].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/c2d73b3d-6771-449f-a8e9-3c458cb9c074?project=870469359193 ].

gcloud builds submit only displays logs from Cloud Storage. To view logs from Cloud Logging, run:
gcloud beta builds submit

Waiting for build to complete. Polling interval: 1 second(s).

BUILD FAILURE: Build step failure: build step 2 "us-central1-docker.pkg.dev/magnetic-guild-473821-t7/mlops-artifacts/mlops-pipeline-image:v1" failed: step exited with non-zero status: 2
ERROR: (gcloud.builds.submit) build c2d73b3d-6771-449f-a8e9-3c458cb9c074 completed with status "FAILURE"


SyntaxError: invalid syntax (1072716569.py, line 1)